In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go

In [3]:
df=pd.read_excel('C:/DATA SCIENCE/PLACEMENT/Buckman/Sample Data for shortlisting.xlsx')
df.head()

,S. No.,City,Gender,Marital Status,Age,Education,Role,Number of investors in family,Household Income,Percentage of Investment,Source of Awareness about Investment,Knowledge level about different investment product,Knowledge level about sharemarket,Knowledge about Govt. Schemes,Investment Influencer,Investment Experience,Risk Level,Return Earned,Reason for Investment
0,1,New York,Men,Never Married,Early Working,Secondary,Marketing and Sales Executive,2,US$ 2736 to US$ 8205,Don't Want to Reveal,Television,7,9,4,Family Reference,Less Than 1 Year,Low,Negative Return,Tax
1,2,New York,Women,Married,Early Working,Middle,Advertising and Promotion Executive,2,US$ 19146 to US$ 24615,Don't Want to Reveal,Workers,7,5,2,Workers Reference,Less Than 1 Year,Low,10 to 12,Others
2,3,Seattle,Women,Married,Early Working,Teritary,Training and Development Executive,2,US$ 2736 to US$ 8205,Above 26%,Family,4,3,2,Friends Reference,Above 9 Years,Low,1 to 4,Return
3,4,San Francisco,Men,Never Married,Elderly,Uneducated,Training and Development Executive,5,US$ 19146 to US$ 24615,16% to 20%,Family,5,1,3,Friends Reference,7 Years to 9 Years,Low,10 to 12,Tax
4,5,San Francisco,Men,Married,Prime Working,Primary,Computer and Information System Executive,2,US$ 13676 to US$ 19145,16% to 20%,Workers,6,10,5,Family Reference,Above 9 Years,Low,5 to 6,Fun and Exitement


In [4]:
df.set_index('S. No.',inplace=True)
df.head()

,City,Gender,Marital Status,Age,Education,Role,Number of investors in family,Household Income,Percentage of Investment,Source of Awareness about Investment,Knowledge level about different investment product,Knowledge level about sharemarket,Knowledge about Govt. Schemes,Investment Influencer,Investment Experience,Risk Level,Return Earned,Reason for Investment
S. No.,,,,,,,,,,,,,,,,,,
1,New York,Men,Never Married,Early Working,Secondary,Marketing and Sales Executive,2,US$ 2736 to US$ 8205,Don't Want to Reveal,Television,7,9,4,Family Reference,Less Than 1 Year,Low,Negative Return,Tax
2,New York,Women,Married,Early Working,Middle,Advertising and Promotion Executive,2,US$ 19146 to US$ 24615,Don't Want to Reveal,Workers,7,5,2,Workers Reference,Less Than 1 Year,Low,10 to 12,Others
3,Seattle,Women,Married,Early Working,Teritary,Training and Development Executive,2,US$ 2736 to US$ 8205,Above 26%,Family,4,3,2,Friends Reference,Above 9 Years,Low,1 to 4,Return
4,San Francisco,Men,Never Married,Elderly,Uneducated,Training and Development Executive,5,US$ 19146 to US$ 24615,16% to 20%,Family,5,1,3,Friends Reference,7 Years to 9 Years,Low,10 to 12,Tax
5,San Francisco,Men,Married,Prime Working,Primary,Computer and Information System Executive,2,US$ 13676 to US$ 19145,16% to 20%,Workers,6,10,5,Family Reference,Above 9 Years,Low,5 to 6,Fun and Exitement


In [5]:
def split_income(value):
    value = value.replace(',', '')  # Remove commas if present
    if ' to ' in value:
        # Split the range and convert to integers
        income_range = value.split(' to ')
        lower_income = int(''.join(char for char in income_range[0] if char.isdigit()))
        upper_income = int(''.join(char for char in income_range[1] if char.isdigit()))
    elif 'Above' in value:
        # Extract the upper limit from "Above" format and set lower limit greater
        upper_limit = int(''.join(char for char in value if char.isdigit()))
        lower_income = upper_limit + 1  # Set lower income greater than upper limit
        upper_income = np.inf
    else:
        # Convert the single value to integer
        lower_income = int(''.join(char for char in value if char.isdigit()))
        upper_income = lower_income  # Set upper income equal to lower income for single value
    return lower_income, upper_income

In [6]:
df[['Lower Income', 'Upper Income']] = df['Household Income'].apply(lambda x: pd.Series(split_income(x)))

print(df)

                 City Gender Marital Status            Age   Education  \
S. No.                                                                   
1            New York    Men  Never Married  Early Working   Secondary   
2            New York  Women        Married  Early Working      Middle   
3             Seattle  Women        Married  Early Working    Teritary   
4       San Francisco    Men  Never Married        Elderly  Uneducated   
5       San Francisco    Men        Married  Prime Working     Primary   
...               ...    ...            ...            ...         ...   
806           Seattle    Men        Married  Early Working   Secondary   
807           Seattle    Men  Never Married       Children   Secondary   
808     San Francisco    Men  Never Married       Children  Uneducated   
809           Memphis  Women        Married       Children  Uneducated   
810          New York    Men  Never Married  Prime Working      Middle   

                                     

In [7]:
mean=df['Upper Income'].mean()
mean_without_inf = df[df['Upper Income'] != np.inf]['Upper Income'].mean()
df['Upper Income'] = np.where((df['Upper Income'] == np.inf) & (mean_without_inf > df['Lower Income']),
                              mean_without_inf, df['Upper Income'])
df['Upper Income']=df['Upper Income'].round(1)
print(df)

                 City Gender Marital Status            Age   Education  \
S. No.                                                                   
1            New York    Men  Never Married  Early Working   Secondary   
2            New York  Women        Married  Early Working      Middle   
3             Seattle  Women        Married  Early Working    Teritary   
4       San Francisco    Men  Never Married        Elderly  Uneducated   
5       San Francisco    Men        Married  Prime Working     Primary   
...               ...    ...            ...            ...         ...   
806           Seattle    Men        Married  Early Working   Secondary   
807           Seattle    Men  Never Married       Children   Secondary   
808     San Francisco    Men  Never Married       Children  Uneducated   
809           Memphis  Women        Married       Children  Uneducated   
810          New York    Men  Never Married  Prime Working      Middle   

                                     

In [8]:
df['Upper Income'] = df['Upper Income'].replace(np.inf, 100000)
print(df)

                 City Gender Marital Status            Age   Education  \
S. No.                                                                   
1            New York    Men  Never Married  Early Working   Secondary   
2            New York  Women        Married  Early Working      Middle   
3             Seattle  Women        Married  Early Working    Teritary   
4       San Francisco    Men  Never Married        Elderly  Uneducated   
5       San Francisco    Men        Married  Prime Working     Primary   
...               ...    ...            ...            ...         ...   
806           Seattle    Men        Married  Early Working   Secondary   
807           Seattle    Men  Never Married       Children   Secondary   
808     San Francisco    Men  Never Married       Children  Uneducated   
809           Memphis  Women        Married       Children  Uneducated   
810          New York    Men  Never Married  Prime Working      Middle   

                                     